In [1]:
from snow_pc.modeling import terrain_models, surface_models
import leafmap

In [19]:
# laz_fp = r'C:\Users\naheemadebisi\Downloads\dev_work\explore_data\20240213_MCS_Pre_1\20240213_v1-240213_190028_Scanner_1.laz'
# dem_fp = r'C:\Users\naheemadebisi\Downloads\dev_work\dem.tif'

In [21]:
# leafmap.view_lidar(laz_fp)

In [20]:
# dtm_laz, dtm_tif = terrain_models(laz_fp, dem_fp = dem_fp)
# dsm_laz, dsm_tif = surface_models(laz_fp, dem_fp = dem_fp) 

## Photogrammetry point clouds

In [3]:
laz_fp = '/SNOWDATA/IDALS/2024/20231113/20231113_MCS_UPS/20231113_UPS_shifted_classified.laz'
dem_fp = '/SNOWDATA/IDALS/REF_DEM/MCS_REFDEM_WGS84.tif'

In [1]:
# leafmap.view_lidar(laz_fp)

In [5]:
dtm_laz, dtm_tif = terrain_models(laz_fp, user_dem = dem_fp, lidar_pc = 'no')
dsm_laz, dsm_tif = surface_models(laz_fp, user_dem = dem_fp, lidar_pc = 'no') 

## DEM filtering analysis for ALS, ULS and UAV


In [3]:
from snow_pc.filtering import return_filtering, elm_filtering, outlier_filtering, dem_filtering, ground_segmentation
import leafmap

In [5]:
def explore_lidar(filename):
    las = leafmap.read_lidar(filename)
    print(las.header.point_count)

### ULS

In [3]:
ULS_laz = '/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ULS/snow-pc/results/unfiltered_merge.laz'
dem_fp = '/SNOWDATA/IDALS/REF_DEM/MCS_REFDEM_WGS84.tif'

In [4]:
explore_lidar(ULS_laz)

18405812


In [5]:
return_filter = return_filtering(ULS_laz)

In [8]:
return_filter

'returns_filtered.laz'

In [6]:
explore_lidar(return_filter)

18405812


In [9]:
dem_filter = dem_filtering('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ULS/snow-pc/results/returns_filtered.laz', 
              user_dem= dem_fp, dem_low= 10, dem_high=50)

In [10]:
explore_lidar(dem_filter)

18405808


In [11]:
elm_filter = elm_filtering('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ULS/snow-pc/results/dem_filtered.laz')

In [12]:
explore_lidar(elm_filter)

18405808


In [13]:
outlier_filter = outlier_filtering('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ULS/snow-pc/results/elm_filtered.laz', multiplier=2.5)

In [14]:
explore_lidar(outlier_filter)

18405808


In [15]:
ground_filter = ground_segmentation('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ULS/snow-pc/results/outlier_filtered.laz')

In [19]:
explore_lidar('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ULS/snow-pc/results/ground_segmented.laz')

11365941


### UPS

In [20]:
UPS_laz = '/SNOWDATA/IDALS/2024/20231113/20231113_MCS_UPS/20231113_UPS_shifted_classified.laz'
dem_fp = '/SNOWDATA/IDALS/REF_DEM/MCS_REFDEM_WGS84.tif'

In [21]:
explore_lidar(UPS_laz)

88972308


In [22]:
return_filter = return_filtering(UPS_laz)

In [23]:
explore_lidar(return_filter)

88972308


In [24]:
dem_filter = dem_filtering(UPS_laz, 
              user_dem= dem_fp, dem_low= 10, dem_high=50)

In [25]:
explore_lidar(dem_filter)

88967547


In [26]:
elm_filter = elm_filtering('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_UPS/dem_filtered.laz')
explore_lidar(elm_filter)

88967547


In [27]:
outlier_filter = outlier_filtering('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_UPS/elm_filtered.laz', multiplier=2.5)
explore_lidar(outlier_filter)

88967547


In [2]:
ground_filter = ground_segmentation('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_UPS/outlier_filtered.laz', lidar_pc= 'no')

In [5]:
explore_lidar('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_UPS/ground_segmented.laz')

58535763


In [1]:
# leafmap.whiteboxgui()

In [2]:
import geopandas as gpd
import json
import subprocess

# Load the shapefile
gdf = gpd.read_file('/SNOWDATA/IDALS/2024/20231113/ULS_Polygon/ULS.shp')

# Convert the first geometry in the shapefile to WKT format
polygon = gdf['geometry'][0].wkt

# Create a PDAL pipeline to clip the LiDAR file
pipeline = {
    "pipeline": [
        "/SNOWDATA/IDALS/2024/20231113/20231113_MCS/ice-road/results/unfiltered_merge.laz",
        {
            "type": "filters.crop",
            "polygon": polygon
        },
        "/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ALS_clip/unfiltered.laz"
    ]
}

# Write the pipeline to a JSON file
with open('pipeline.json', 'w') as f:
    json.dump(pipeline, f)

# Run the PDAL pipeline
subprocess.run(['pdal', 'pipeline', 'pipeline.json'])

CompletedProcess(args=['pdal', 'pipeline', 'pipeline.json'], returncode=0)

### ALS

In [6]:
ALS_laz = '/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ALS_clip/unfiltered.laz'
dem_fp = '/SNOWDATA/IDALS/REF_DEM/MCS_REFDEM_WGS84.tif'
explore_lidar(ALS_laz)

8633780


In [7]:
return_filter = return_filtering(ALS_laz)
explore_lidar(return_filter)

8633780


In [13]:
dem_filter = dem_filtering('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ALS_clip/returns_filtered.laz', 
              user_dem= dem_fp, dem_low= 10, dem_high=30)
explore_lidar(dem_filter)

8581131


In [14]:
elm_filter = elm_filtering('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ALS_clip/dem_filtered.laz')
explore_lidar(elm_filter)

8581131


In [15]:
outlier_filter = outlier_filtering('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ALS_clip/elm_filtered.laz', multiplier=2.5)
explore_lidar(outlier_filter)

8581131


In [16]:
ground_filter = ground_segmentation('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ALS_clip/outlier_filtered.laz')

In [17]:
explore_lidar('/SNOWDATA/IDALS/2024/20231113/20231113_MCS_ALS_clip/ground_segmented.laz')

5480903
